# 4.5
试编程实现基于对率回归进行划分选择的决策树算法，并为表4.3中数据生成一棵决策树。

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from ch4 import TreeGenerate_logistic

In [3]:
dataset = pd.read_csv('data/table_4_3_watermelon_3_0_num.csv')
dataset = dataset.drop('Idx',axis=1)

In [23]:
class Node(object):
    def __init__(self, att=None, child={}, label=None, weight = None):
        self.att = att
        self.label = label 
        self.child = child
        self.w = weight
        
def mode(l):      
    #统计list中各个数值出现的次数  
    count_dict=dict()
    for i in l:  
        if i in count_dict.keys():  
            count_dict[i]+=1 
        else:  
            count_dict[i]=1
    #求出现次数的最大值  
    max_appear=0  
    for v in count_dict.values():  
        if v>max_appear:  
            max_appear=v
    mode_list=[]
    for k,v in count_dict.items():  
        if v==max_appear:  
            mode_list.append(k)
    return mode_list

def get_one_label(label):
    mlabel = mode(label)
    if len(mlabel) > 1:
        mlabel = mlabel[np.random.randint(len(mlabel))]
    else:
        mlabel = mlabel[0]
    return mlabel

In [44]:
def TreeGenerate_logistic(dataset):
    att = list(dataset.columns[:-1])
#     要有初始化值(None,{},None)。
    rn = Node(None, {}, None)
#     if D中样本全属于同一类别
    if len(np.unique(dataset.ix[:,-1])) == 1:
        rn.label = np.unique(dataset.ix[:,-1])[0]
        return rn
#     if D中样本在A上取值相同
    same = 0
    for a in att:
        if len(np.unique(dataset.ix[:,a]))>1:
            same = 1
            break
    if same==0:
        mlabel = mode(dataset.ix[:,-1])
        rn.label = mlabel
        return rn
#     构建logistic回归
    rn.label = get_one_label(dataset.ix[:,-1])    
    log_model = LogisticRegression() 
    log_model.fit(dataset.iloc[:,:-1], dataset.iloc[:,-1]) 
    predict_label = log_model.predict(dataset.iloc[:,:-1]) 
    rn.w = np.hstack([log_model.coef_[0], log_model.intercept_])
    left_value = 'y=0'
    right_value = 'y=1'
    if dataset.ix[predict_label<=0,:].empty or len(np.unique(predict_label))==1:
        rn.child[left_value] = Node(None, {}, None)
        rn.child[left_value].label = 0
        
    else:
        rn.child[left_value] = TreeGenerate_logistic(dataset.ix[predict_label<=0,:]) 
    if dataset.ix[predict_label>0,:].empty or len(np.unique(predict_label))==1:
        rn.child[right_value] = Node(None, {}, None)
        rn.child[right_value].label = 1
    else:
        rn.child[right_value] = TreeGenerate_logistic(dataset.ix[predict_label>0,:]) 
    return rn

In [45]:
rn = TreeGenerate_logistic(dataset)

In [46]:
rn.w

array([ 0.10205268, -0.37317245, -0.4981436 , -1.09623118, -0.45255551,
        0.18352238,  0.43046137,  0.2955575 ,  0.69335734])

In [48]:
rn.child

{'y=0': <__main__.Node at 0x15c0056fd0>,
 'y=1': <__main__.Node at 0x15c004b5f8>}

In [50]:
rn.child['y=0'].w

array([-0.1654895 , -0.07651756, -0.65796895, -0.45010717, -0.2973238 ,
        0.45282301, -0.2186481 , -0.03678544, -0.30481051])

In [51]:
rn.child['y=1'].w

array([ 0.21762188, -0.08658134,  0.31408021,  0.        , -0.08658134,
       -0.32902918,  0.58208979,  0.17547878,  0.80344473])

In [56]:
rn.child['y=0'].child['y=0'].child

{}